In [1]:
import os
import torch
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_dtype(torch.float64)

import botorch
from botorch.acquisition import ExpectedImprovement, NoisyExpectedImprovement,\
    PosteriorMean, UpperConfidenceBound
from botorch.optim import optimize_acqf
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition.monte_carlo import qExpectedImprovement, qProbabilityOfImprovement, qUpperConfidenceBound
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.exceptions import BadInitialCandidatesWarning

import time
import warnings
from botorch.acquisition.objective import ConstrainedMCObjective
from botorch.test_functions import Hartmann, Ackley, EggHolder, Cosine8, DixonPrice, Griewank, Levy, Powell

from botorch.models import SingleTaskGP, FixedNoiseGP, ModelListGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.kernels import MaternKernel

from scipy.stats.qmc import Sobol
import pandas as pd

# pose as a maximization problem for all
test_functions = {
    'sinc_plus_2':{
        'function':lambda x: torch.sinc(x + 2.),
        'bounds':torch.tensor([-20., 20.]).unsqueeze(-1)
    },
    'hartmann':{
        'function':Hartmann(negate=True),
        'bounds':torch.tensor([[0] * 6, [1] * 6]),
        'maximum_value':torch.tensor(-3.32237),
        'maximum_point':torch.tensor([[0.20169, 0.150011, 0.476874, 0.275332, 0.311652, 0.6573]])
    },
    'ackley':{
        'function':Ackley(negate=True),
        'bounds':torch.tensor([[-29.768] * 2, [32.768] * 2]),
        'maximum_value':torch.tensor(0),
        'maximum_point':torch.tensor([[0,0]])
    },
    'eggholder':{
        'function':EggHolder(negate=True),
        'bounds':torch.tensor([[-512] * 2, [512] * 2]),
        'maximum_value':torch.tensor(959.6407),
        'maximum_point':torch.tensor([[512, 404.2319]])
    },
    'cosine':{
        'function':Cosine8(negate=False),
        'bounds':torch.tensor([[-1] * 8, [0.7] * 8]),
        'maximum_value':torch.tensor(0.8),
        'maximum_point':torch.tensor([[0,0,0,0,0,0,0,0]])
    },
    'dixonprice':{
        'function':DixonPrice(dim=8, negate=True),
        'bounds':torch.tensor([[-10] * 8, [10] * 8]),
        'maximum_value':torch.tensor(0),
#         'maximum_point':torch.tensor([[1, ]]) # too complicated
    },
    'griewank':{
        'function':Griewank(dim=8, negate=True),
        'bounds':torch.tensor([[-600] * 8,[500] * 8]),
        'maximum_value':torch.tensor(0),
        'maximum_point':torch.tensor([[0,0,0,0,0,0,0,0]])
    },
    'levy':{
        'function':Levy(dim=8),
        'bounds':torch.tensor([[-10] * 8, [10] * 8]),
        'maximum_value':torch.tensor(0),
        'maximum_point':torch.tensor([[1,1,1,1,1,1,1,1]])
    },
    'powell':{
        'function':Powell(dim=4, negate=True),
        'bounds':torch.tensor([[-4] * 8, [5] * 8]),
        'maximum_value':torch.tensor(0),
        'maximum_point':torch.tensor([[0,0,0,0,0,0,0,0]])
    }
}
for k in test_functions.keys():
    test_functions[k]['bounds'] = test_functions[k]['bounds'].double()
    
models = {
    'matern2_5':{
        'gp':SingleTaskGP,
        'likelihood':ExactMarginalLogLikelihood,
        'covar':MaternKernel(2.5) # Matern by default
    },
    'matern1_5':{
        'gp':SingleTaskGP,
        'likelihood':ExactMarginalLogLikelihood,
        'covar':MaternKernel(nu=1.5) # Matern by default
    },
    'matern0_5':{
        'gp':SingleTaskGP,
        'likelihood':ExactMarginalLogLikelihood,
        'covar':MaternKernel(nu=0.5) # Matern by default
    }
}

def initialize_first_batch(bounds, m=3):
    """
    bounds: the range in which we want to generate points.
    m: the power of 2 representing the number of Sobol points to generate.
    """
    d = bounds.shape[1]
    sobol_points = torch.tensor(Sobol(d=d, scramble=False).random_base2(m=m))
    # map each dimension to the bounds
    j = 0
    for i in bounds.T:
        sobol_points.T[j] = (i[1] - i[0]) * sobol_points.T[j] + i[0]
        j += 1
    return sobol_points

def prep_model(models, model_key, X_train, y_train, state_dict=None):
    """
    Initialize model according to specifications from the model dict and training data
    Hotload the params from state_dict to speed up likelihood fitting
    """
    model = models[model_key]['gp'](X_train, y_train, covar_module=models[model_key]['covar'])
    if state_dict is not None:
        model.load_state_dict(state_dict)
        
    mll = models[model_key]['likelihood'](model.likelihood, model)
    return mll, model

def training_loop(test_functions, function_key, 
                  models, model_key, acq_function,
                  loops, initialization_count_m=2, q=4):
    """
    Trains specified model on function defined by specified key for specified loop count
    """
    # initialize data
    rv = test_functions[function_key]['function']
    X_train = initialize_first_batch(test_functions[function_key]['bounds'], m=initialization_count_m)
    y_train = rv(X_train)
    if len(y_train.shape) == 1:
        y_train = y_train.unsqueeze(-1)
    mll, model = prep_model(models=models, model_key=model_key, X_train=X_train, y_train=y_train)
    
    # fit generate update loop
    for i in range(loops):
        # reload model
        mll, model = prep_model(models=models, 
            model_key=model_key, 
            X_train=X_train, 
            y_train=y_train,
            state_dict=model.state_dict())
        # fit model and acquisition function
        fit_gpytorch_mll(mll)
        best_value = y_train.max()
        
        if acq_function == qExpectedImprovement:
            acq = acq_function(model=model, best_f=best_value, maximize=True)
        if acq_function == qProbabilityOfImprovement:
            acq = acq_function(model=model, best_f=best_value)
        if acq_function == qUpperConfidenceBound:
            acq = acq_function(model=model, beta=0.2)
            
        # generate new points
        candidates, _ = optimize_acqf(
            acq_function=acq,
            bounds=test_functions[function_key]['bounds'],
            q=q,
            num_restarts=10,
            raw_samples=20,  # used for initialization heuristic
        )
        new_y = rv(candidates)
        if len(new_y.shape) == 1:
            new_y = new_y.unsqueeze(-1)
        X_train = torch.cat([X_train, candidates])
        y_train = torch.cat([y_train, new_y])
        best_value = y_train.max()
        
    return mll, model, X_train, y_train, best_value

In [2]:
performance = []

for model_key in models.keys():
    for function_key in test_functions.keys():
        for q in [1, 4, 16]:  
            for acq_function in [qExpectedImprovement, qProbabilityOfImprovement, qUpperConfidenceBound]:
                if acq_function == qExpectedImprovement:
                    acq_function_name = 'Expected Improvement'
                if acq_function == qProbabilityOfImprovement:
                    acq_function_name = 'Probability of Improvement'
                if acq_function == qUpperConfidenceBound:
                    acq_function_name = 'Upper Confidence Bound'
                
                print('attempting - function {} model {} q {} acq {}'\
                    .format(function_key, model_key, q, acq_function))
                loops = 256 // q
                mll, model, X_train, y_train, best_value = training_loop(test_functions, function_key=function_key,
                      models=models, model_key=model_key, acq_function=acq_function,
                      loops=loops, q=q)
                
                performance.append({
                    'Model':model_key,
                    'Test Function':function_key,
                    'Q':q,
                    'Acquisition Function':acq_function_name,
                    'Performance':float(best_value)
                })
                print(best_value)
                print('')

attempting - function sinc_plus_2 model matern2_5 q 1 acq <class 'botorch.acquisition.monte_carlo.qExpectedImprovement'>
tensor(0.9790)

attempting - function sinc_plus_2 model matern2_5 q 1 acq <class 'botorch.acquisition.monte_carlo.qProbabilityOfImprovement'>
tensor(0.0190)

attempting - function sinc_plus_2 model matern2_5 q 1 acq <class 'botorch.acquisition.monte_carlo.qUpperConfidenceBound'>
tensor(0.0155)

attempting - function sinc_plus_2 model matern2_5 q 4 acq <class 'botorch.acquisition.monte_carlo.qExpectedImprovement'>
tensor(0.9881)

attempting - function sinc_plus_2 model matern2_5 q 4 acq <class 'botorch.acquisition.monte_carlo.qProbabilityOfImprovement'>


/Users/eriqqc/workspace/gaussian_process_benchmarks/botorch/botorch/optim/optimize.py:306: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2.'), NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal')]
Trying again with a new set of initial conditions.
  warnings.warn(first_warn_msg, RuntimeWarning)


tensor(0.9631)

attempting - function sinc_plus_2 model matern2_5 q 4 acq <class 'botorch.acquisition.monte_carlo.qUpperConfidenceBound'>


/Users/eriqqc/workspace/gaussian_process_benchmarks/botorch/botorch/optim/optimize.py:306: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2.'), NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal')]
Trying again with a new set of initial conditions.
  warnings.warn(first_warn_msg, RuntimeWarning)
/Users/eriqqc/workspace/gaussian_process_benchmarks/botorch/botorch/optim/optimize.py:306: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2.'), NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal')]
Trying again with a new set of initial conditions.
  wa

/Users/eriqqc/workspace/gaussian_process_benchmarks/botorch/botorch/optim/optimize.py:306: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2.'), NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal')]
Trying again with a new set of initial conditions.
  warnings.warn(first_warn_msg, RuntimeWarning)
/Users/eriqqc/workspace/gaussian_process_benchmarks/botorch/botorch/optim/optimize.py:328: RuntimeWarning: Optimization failed on the second try, after generating a new set of initial conditions.
  warnings.warn(


...

/Users/eriqqc/workspace/gaussian_process_benchmarks/botorch/botorch/optim/optimize.py:306: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2.'), NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal')]
Trying again with a new set of initial conditions.
  warnings.warn(first_warn_msg, RuntimeWarning)


tensor(0.7465)

attempting - function hartmann model matern0_5 q 16 acq <class 'botorch.acquisition.monte_carlo.qExpectedImprovement'>
tensor(2.4424)

attempting - function hartmann model matern0_5 q 16 acq <class 'botorch.acquisition.monte_carlo.qProbabilityOfImprovement'>
tensor(2.5521)

attempting - function hartmann model matern0_5 q 16 acq <class 'botorch.acquisition.monte_carlo.qUpperConfidenceBound'>


/Users/eriqqc/workspace/gaussian_process_benchmarks/botorch/botorch/optim/optimize.py:306: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2.'), NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal')]
Trying again with a new set of initial conditions.
  warnings.warn(first_warn_msg, RuntimeWarning)


tensor(0.7465)

attempting - function ackley model matern0_5 q 1 acq <class 'botorch.acquisition.monte_carlo.qExpectedImprovement'>
tensor(-4.8125)

attempting - function ackley model matern0_5 q 1 acq <class 'botorch.acquisition.monte_carlo.qProbabilityOfImprovement'>
tensor(-5.2478)

attempting - function ackley model matern0_5 q 1 acq <class 'botorch.acquisition.monte_carlo.qUpperConfidenceBound'>
tensor(-2.1502)

attempting - function ackley model matern0_5 q 4 acq <class 'botorch.acquisition.monte_carlo.qExpectedImprovement'>
tensor(-5.4240)

attempting - function ackley model matern0_5 q 4 acq <class 'botorch.acquisition.monte_carlo.qProbabilityOfImprovement'>
tensor(-7.5340)

attempting - function ackley model matern0_5 q 4 acq <class 'botorch.acquisition.monte_carlo.qUpperConfidenceBound'>
tensor(-6.2881)

attempting - function ackley model matern0_5 q 16 acq <class 'botorch.acquisition.monte_carlo.qExpectedImprovement'>
tensor(-7.5340)

attempting - function ackley model mater

tensor(-30.3125)

attempting - function powell model matern0_5 q 16 acq <class 'botorch.acquisition.monte_carlo.qProbabilityOfImprovement'>
tensor(-30.3125)

attempting - function powell model matern0_5 q 16 acq <class 'botorch.acquisition.monte_carlo.qUpperConfidenceBound'>
tensor(-30.3125)



In [3]:
print(performance)

[{'Model': 'matern2_5', 'Test Function': 'sinc_plus_2', 'Q': 1, 'Acquisition Function': 'Expected Improvement', 'Performance': 0.9790379869855272}, {'Model': 'matern2_5', 'Test Function': 'sinc_plus_2', 'Q': 1, 'Acquisition Function': 'Probability of Improvement', 'Performance': 0.01897608328852435}, {'Model': 'matern2_5', 'Test Function': 'sinc_plus_2', 'Q': 1, 'Acquisition Function': 'Upper Confidence Bound', 'Performance': 0.01552081689641406}, {'Model': 'matern2_5', 'Test Function': 'sinc_plus_2', 'Q': 4, 'Acquisition Function': 'Expected Improvement', 'Performance': 0.9880810856966564}, {'Model': 'matern2_5', 'Test Function': 'sinc_plus_2', 'Q': 4, 'Acquisition Function': 'Probability of Improvement', 'Performance': 0.9630629127866782}, {'Model': 'matern2_5', 'Test Function': 'sinc_plus_2', 'Q': 4, 'Acquisition Function': 'Upper Confidence Bound', 'Performance': 0.8608320234819088}, {'Model': 'matern2_5', 'Test Function': 'sinc_plus_2', 'Q': 16, 'Acquisition Function': 'Expected I

In [4]:
with pd.option_context('display.max_rows', None):
    display(pd.DataFrame(performance))

,Model,Test Function,Q,Acquisition Function,Performance
0,matern2_5,sinc_plus_2,1,Expected Improvement,9.790380e-01
1,matern2_5,sinc_plus_2,1,Probability of Improvement,1.897608e-02
2,matern2_5,sinc_plus_2,1,Upper Confidence Bound,1.552082e-02
3,matern2_5,sinc_plus_2,4,Expected Improvement,9.880811e-01
4,matern2_5,sinc_plus_2,4,Probability of Improvement,9.630629e-01
5,matern2_5,sinc_plus_2,4,Upper Confidence Bound,8.608320e-01
6,matern2_5,sinc_plus_2,16,Expected Improvement,9.035598e-01
7,matern2_5,sinc_plus_2,16,Probability of Improvement,9.863380e-01
8,matern2_5,sinc_plus_2,16,Upper Confidence Bound,9.267292e-01
9,matern2_5,hartmann,1,Expected Improvement,2.310047e+00


In [5]:
pd.DataFrame(performance).to_csv('results.csv', index=False)

In [6]:
pd.DataFrame(performance).pivot(index=['Model', 'Q', 'Acquisition Function'], 
                                columns='Test Function', 
                                values='Performance')

Test Function                              ackley    cosine  dixonprice  \
Model     Q  Acquisition Function                                         
matern0_5 1  Expected Improvement       -4.812498 -0.426596        -1.0   
             Probability of Improvement -5.247782 -0.464658        -1.0   
             Upper Confidence Bound     -2.150220 -0.203534        -1.0   
          4  Expected Improvement       -5.424006 -0.267279        -1.0   
             Probability of Improvement -7.534038 -0.323844        -1.0   
             Upper Confidence Bound     -6.288126 -0.745685        -1.0   
          16 Expected Improvement       -7.534038 -0.414881        -1.0   
             Probability of Improvement -4.479885 -0.391781        -1.0   
             Upper Confidence Bound     -5.651867 -0.745685        -1.0   
matern1_5 1  Expected Improvement       -2.615430 -0.130287        -1.0   
             Probability of Improvement -6.625989 -0.038398        -1.0   
             Upper Confidence Bound     -7.534038 -0.385963        -1.0   
          4  Expected Improvement       -7.534038 -0.417306        -1.0   
             Probability of Improvement -6.310580 -0.211645        -1.0   
             Upper Confidence Bound     -7.534038 -0.744938        -1.0   
          16 Expected Improvement       -4.524579 -0.247465        -1.0   
             Probability of Improvement -5.494064 -0.340707        -1.0   
             Upper Confidence Bound     -6.087604 -0.577665        -1.0   
matern2_5 1  Expected Improvement       -4.180610 -0.268585        -1.0   
             Probability of Improvement -7.534038  0.042410        -1.0   
             Upper Confidence Bound     -7.534038 -0.399936        -1.0   
          4  Expected Improvement       -7.534038 -0.474461        -1.0   
             Probability of Improvement -6.190177 -0.142833        -1.0   
             Upper Confidence Bound     -2.797448 -0.414237        -1.0   
          16 Expected Improvement       -7.534038 -0.364550        -1.0   
             Probability of Improvement -1.087403 -0.650348        -1.0   
             Upper Confidence Bound     -5.803360 -0.353581        -1.0   

Test Function                             eggholder  griewank  hartmann  \
Model     Q  Acquisition Function                                         
matern0_5 1  Expected Improvement        807.528242  -6.00159  2.241206   
             Probability of Improvement  911.784328  -6.00159  2.257121   
             Upper Confidence Bound      732.581616  -6.00159  3.073004   
          4  Expected Improvement        888.650574  -6.00159  2.092582   
             Probability of Improvement  744.756499  -6.00159  2.781128   
             Upper Confidence Bound      936.695988  -6.00159  0.746493   
          16 Expected Improvement        888.695180  -6.00159  2.442387   
             Probability of Improvement  802.505953  -6.00159  2.552089   
             Upper Confidence Bound      839.051077  -6.00159  0.746493   
matern1_5 1  Expected Improvement        879.231101  -6.00159  1.825018   
             Probability of Improvement  820.487134  -6.00159  2.678764   
             Upper Confidence Bound      934.772976  -6.00159  3.040563   
          4  Expected Improvement        803.799186  -6.00159  2.004992   
             Probability of Improvement  748.173457  -6.00159  2.365491   
             Upper Confidence Bound      920.196100  -6.00159  1.172294   
          16 Expected Improvement        785.293729  -6.00159  2.579540   
             Probability of Improvement  886.951887  -6.00159  1.690636   
             Upper Confidence Bound      868.753200  -6.00159  2.461558   
matern2_5 1  Expected Improvement        882.142015  -6.00159  2.310047   
             Probability of Improvement  851.737919  -6.00159  2.145831   
             Upper Confidence Bound      915.143731  -6.00159  2.925541   
          4  Expected Improvement        864.573705  -6.00159  1.748551   
             Probability